In [1]:
import pandas as pd
import numpy as np
import openpyxl
from pandas import ExcelWriter
import xlsxwriter

In [ ]:
fileH = (r"C:\Users\j\.xlsx")

In [68]:
datframeh = pd.read_excel(fileH, header= 2)


In [ ]:
datframeh.head

In [19]:
datframeh.columns
# datframeh.info
# datframeh.head()
# len(datframeh)
# datframeh.isnull().values.any()
# datframe1 = datframe.dropna(axis=1, how="all") # # Not needed. HVAC WO have no columns fully Null. Only drop columns where all values are NaN

Index(['Order #', 'Web Request #', 'Description', 'Order Type', 'Order Status',
       'Work Status', 'Address', 'Location Name', 'Client Division',
       'Requester', 'Service Provider', 'Priority', 'Created Date',
       'Completed Date'],
      dtype='object')

In [70]:
patternbas1 = "bas |automation|temperature|cold|hot|cool|heat|setpoint|set point|circulation|schedule|alarm"
maskbas1 = datframeh['Description'].str.contains( patternbas1,case=False, na=True)
# non bas removed
bask = datframeh[maskbas1]
demands = "ZF05"
demandonly1 = bask["Order Type"].str.contains(demands, case=False, na=True)
# preventative remove
demandso = bask[demandonly1]
# demandso.head
# demandso = pd.DataFrame(datframe1[mask1])

In [ ]:
demandso.info

In [ ]:
BASlocations = "building1"|building2..."

In [72]:
basloc = demandso["Address"].str.contains(BASlocations, case=False, na=True)
basdemandh = demandso[basloc]
# basdemandh.head

In [ ]:
remotelocations = "remotely-accessible-building1|remotely-accessible-building2..."

In [ ]:
remoteloc = basdemandh["Address"].str.contains(remotelocations, case=False, na=True)
basRemote= basdemandh[remoteloc]
basRemote.head

In [75]:
bas_loc_count = basdemandh["Address"].value_counts().reset_index()
bas_loc_count.columns = ['Location', 'count']

bas_div_count = basdemandh["Client Division"].value_counts().reset_index()
bas_div_count.columns = ['Division', 'count']

In [80]:
basdemandh.columns

Index(['Order #', 'Web Request #', 'Description', 'Order Type', 'Order Status',
       'Work Status', 'Address', 'Location Name', 'Client Division',
       'Requester', 'Service Provider', 'Priority', 'Created Date',
       'Completed Date'],
      dtype='object')

In [ ]:
#Location and Division by internal or external
bas_loc_prov = basdemandh.groupby(['Address', 'Service Provider']).size().reset_index(name='count')
per_loc_prov= bas_loc_prov.pivot_table(index='Address', columns= ['Service Provider'], values='count', fill_value=0).reset_index()
per_loc_prov_df = pd.DataFrame(per_loc_prov)

bas_div_prov = basdemandh.groupby(['Client Division', 'Service Provider']).size().reset_index(name='count')
per_div_prov= bas_div_prov.pivot_table(index='Client Division', columns= ['Service Provider'], values='count', fill_value=0).reset_index()
per_div_prov_df = pd.DataFrame(per_div_prov)

# import os
# os.chdir(path=r"C:\Users\\Temp")
# per_loc_count_df.to_excel('HVACperDivision.xlsx', index=False)

# Location and Division by Priority
bas_loc_prio = basdemandh.groupby(['Address', 'Priority']).size().reset_index(name='count')
per_loc_prio= bas_loc_prio.pivot_table(index='Address', columns= ['Priority'], values='count', fill_value=0).reset_index()
per_loc_prio_df = pd.DataFrame(per_loc_prio)

bas_div_prio = basdemandh.groupby(['Client Division', 'Priority']).size().reset_index(name='count')
per_div_prio= bas_div_prio.pivot_table(index='Client Division', columns= ['Priority'], values='count', fill_value=0).reset_index()
per_div_prio_df = pd.DataFrame(per_div_prio)

In [ ]:
# NOT ASSIGNED COULD BE CANCELLED SO SORT BY CANCELLED/ STATUS AS WELL NEXT TIME

In [ ]:
with pd.ExcelWriter('HVAC_BreakdownV1.xlsx', engine='xlsxwriter') as writer:
    bask.to_excel(writer, sheet_name='TAB BAS Related', index=False)
    demandso.to_excel(writer, sheet_name='TAB BAS Demand', index=False)
    basdemandh.to_excel(writer, sheet_name='TAB BAS Demand REAL', index=False)
    basRemote.to_excel(writer, sheet_name='TAB BAS Remote', index=False)
    bas_loc_count.to_excel(writer, sheet_name='Locations Breakdown', index=False)
    bas_div_count.to_excel(writer, sheet_name='Division Breakdown', index=False)
    per_loc_prov_df.to_excel(writer, sheet_name='Location Provider Breakdown', index=False)
    per_loc_prio_df.to_excel(writer, sheet_name='Location Priority Breakdown', index=False)
    per_div_prov_df.to_excel(writer, sheet_name='Division Provider Breakdown', index=False)
    per_div_prio_df.to_excel(writer, sheet_name='Division Priority Breakdown', index=False)
#     othermask.to_excel(writer, sheet_name='Other Issues', index=False)
# print("Excel file 'multi_sheet_output.xlsx' created with multiple sheets.")

Old Notes Functions for filtering top 10

In [ ]:
filtered_df = datframe[(datframe['Remotely Acessible'] != 0) & (datframe['Remotely Acessible'].notnull())]

In [ ]:
value_counts = filtered_df['Location Name'].value_counts()
top_10_c_values = value_counts.head(10)
top_10_df = top_10_c_values.reset_index()
top_10_df.columns = ['Location Name', 'frequency']

In [28]:
print(top_10_df)

                              Location Name  frequency
0                       POLICE HEADQUARTERS         29
1                   NORTH YORK CIVIC CENTRE         22
2                                 CITY HALL         17
3  FIRE & PARAMEDIC HQ & EMS STATION NO. 53         17
4    TORONTO EAST COURT & MUNICIPAL OFFICES         16
5  TORONTO PUBLIC HEALTH HEADQUARTERS BUILD         14
6                     REXDALE COMMUNITY HUB         14
7                         YORK CIVIC CENTRE         13
8                  SCARBOROUGH CIVIC CENTRE         12
9         CORPORATE OFFICE BUILDING (HYDRO)         11


In [29]:
top_10_df.to_excel('output10.xlsx', index=False)